In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
import json

/Users/yash/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

# Load intents from intent.json
with open('intent.json', 'r') as f:
    data = json.load(f)  # Load the entire JSON file

# Extract the intents list from the outer "intents" key

intents = data.get("intents", [])

print(intents)

[{'intent': 'ShowMenu', 'examples': ['Can I see the dinner menu?', 'Show me the breakfast options.', "What's available for lunch?", "I'd like to see the dessert menu.", 'What drinks are on the menu?', 'Do you have any appetizers?']}, {'intent': 'PlaceOrder', 'examples': ['I would like to order a pizza.', 'Can I get a salad?', "I'd like to order two burgers.", 'I want to place an order for room service.', 'Can I order a bottle of wine?', 'Please send a steak to my room.']}, {'intent': 'CancelOrder', 'examples': ['I need to cancel my order.', 'Please cancel the order for pasta.', 'Can you cancel my dessert order?', 'I want to cancel my drink order.', "I'd like to stop my room service request."]}, {'intent': 'ModifyOrder', 'examples': ['Can I change my order?', 'I want to add a side of fries to my order.', "I'd like to modify my order to include a drink.", 'Can you remove the dessert from my order?', 'I need to update my order.']}, {'intent': 'SpecialRequests', 'examples': ['Can I get ext

In [12]:

# Check if intents were loaded correctly
if not intents:
    print("No intents found in the file!")

# Prepare texts and labels from the loaded JSON data
texts = []
labels = []
for intent in intents:
    if 'examples' in intent and 'intent' in intent:  # Ensure keys exist
        for example in intent['examples']:
            texts.append(example)
            labels.append(intent['intent'])
    else:
        print("Invalid structure in intent:", intent)


In [14]:

# Encode labels to integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Save label encoder for later use
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Tokenizer and model preparation
MAX_LEN = 64
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(encoded_labels)))


/Users/yash/Library/Python/3.12/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:

# Define custom Dataset class
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [16]:

# Create Dataset and DataLoader
dataset = IntentDataset(texts, encoded_labels, tokenizer, max_len=MAX_LEN)
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Set up training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)


/Users/yash/Library/Python/3.12/lib/python/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:

# Function to train the model
def train_epoch(model, data_loader, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        logits = outputs.logits
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

In [26]:

# Training loop
EPOCHS = 20
for epoch in range(EPOCHS):
    accuracy, loss = train_epoch(model, data_loader, optimizer, device)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Accuracy: {accuracy}, Loss: {loss}')

# Save model using pickle
with open('./Pickle Files/intent_model.pkl', 'wb') as f:
    pickle.dump(model, f)


Epoch 1/20, Accuracy: 1.0, Loss: 0.23828435192505518
Epoch 2/20, Accuracy: 1.0, Loss: 0.21253727003932
Epoch 3/20, Accuracy: 1.0, Loss: 0.19007027397553125
Epoch 4/20, Accuracy: 1.0, Loss: 0.1828965743382772
Epoch 5/20, Accuracy: 1.0, Loss: 0.160817580918471
Epoch 6/20, Accuracy: 1.0, Loss: 0.15362074847022691
Epoch 7/20, Accuracy: 1.0, Loss: 0.14295338715116182
Epoch 8/20, Accuracy: 1.0, Loss: 0.1333911387870709
Epoch 9/20, Accuracy: 1.0, Loss: 0.12532511291404566
Epoch 10/20, Accuracy: 1.0, Loss: 0.11603035281101863
Epoch 11/20, Accuracy: 1.0, Loss: 0.11305923325320084
Epoch 12/20, Accuracy: 1.0, Loss: 0.10617491416633129
Epoch 13/20, Accuracy: 1.0, Loss: 0.09753741199771564
Epoch 14/20, Accuracy: 1.0, Loss: 0.09106385335326195
Epoch 15/20, Accuracy: 1.0, Loss: 0.08846532180905342
Epoch 16/20, Accuracy: 1.0, Loss: 0.0830174870789051
Epoch 17/20, Accuracy: 1.0, Loss: 0.08127857434252898
Epoch 18/20, Accuracy: 1.0, Loss: 0.07587949310739835
Epoch 19/20, Accuracy: 1.0, Loss: 0.070387444

In [27]:

# Example of inference
def predict_intent(text, model, tokenizer, label_encoder, max_len=64):
    model.eval()
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1)
    intent = label_encoder.inverse_transform(preds.cpu().numpy())

    return intent[0]

In [32]:

# Load label encoder and make predictions
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

example_text = "Does it contain any nuts?"
predicted_intent = predict_intent(example_text, model, tokenizer, label_encoder)
print(f'Predicted intent: {predicted_intent}')


Predicted intent: AllergenInquiry
